In [ ]:
import requests # type: ignore
from urllib.parse import urljoin
import pandas as pd # type: ignore
import schedule # type: ignore
import time
from datetime import datetime, timedelta
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email import encoders
from dotenv import load_dotenv


In [18]:
load_dotenv(override=True)
api_username = os.getenv('API_USERNAME')
api_password = os.getenv('API_PASSWORD')
api_url = os.getenv("API_URL")

# VENCIMENTO DE GCP (MODELAGEM E FUNÇÕES DE ENVIO)

In [3]:
auth_url = urljoin(api_url, "/sessions")
body = {
    "nome": api_username,
    "password":api_password
}

response = requests.post(auth_url, json=body)
token = response.json()["token"]
auth_token = "Bearer " + token

# Requisição para obter os dados do Pessoas
pessoas_url = urljoin(api_url, "/pessoas?nested=true")
headers = {"Authorization": auth_token}

response = requests.get(pessoas_url, headers=headers)
pessoas = response.json()
pessoas = pd.DataFrame(pessoas)

pessoas.head(5)

,id,ds_nome,co_tipo_gn,co_equipe_gn,co_formacao_maior_nivel_gn,cv_ingles,cv_portugues,dt_ultimo_certificado_gcp,registro_profissional,declaracao_confidencialidade,...,equipe_padrao,equipe_padrao_kits,equipe_padrao_farmacia,equipe_padrao_naocega,co_externo,dados_co_tipo_gn,dados_co_equipe_gn,dados_co_formacao_maior_nivel_gn,dados_centro,dados_pessoa_especialidade
0,192,Emerson Aredes,704.0,None,31.0,None,None,None,,None,...,1,0.0,0.0,0.0,None,"{'id': 704, 'ds_descricao': 'Suporte TI'}",None,"{'id': 31, 'ds_descricao': 'Mestrado'}",None,[]
1,196,Kleber Giungi,562.0,None,NaN,None,None,None,,None,...,1,1.0,1.0,0.0,None,"{'id': 562, 'ds_descricao': 'Equipe padrão (ad...",None,None,None,[]
2,198,Suporte Polo Trial,664.0,None,NaN,None,None,None,None,None,...,0,0.0,0.0,0.0,None,"{'id': 664, 'ds_descricao': 'Suporte'}",None,None,None,[]
3,199,Dra. Caroline Cândida Carvalho de Oliveira,387.0,None,30.0,None,None,2020-09-28,202.285 SP,None,...,1,0.0,0.0,0.0,None,"{'id': 387, 'ds_descricao': 'Pesquisador Princ...",None,"{'id': 30, 'ds_descricao': 'Especialização'}","{'id': 3, 'descricao': 'Leforte HMCG'}","[{'id': 3, 'co_pessoa': 199, 'co_pessoa_especi..."
4,200,Dra. Tânia Caltabiano,387.0,None,29.0,None,None,None,,None,...,0,0.0,0.0,0.0,None,"{'id': 387, 'ds_descricao': 'Pesquisador Princ...",None,"{'id': 29, 'ds_descricao': 'Superior completo'}","{'id': 3, 'descricao': 'Leforte HMCG'}","[{'id': 4, 'co_pessoa': 200, 'co_pessoa_especi..."


In [4]:
gcp = pessoas[['dados_centro', 'dt_ultimo_certificado_gcp', 'ds_nome','dados_co_tipo_gn']].copy()
gcp.loc[:, 'dados_centro_id'] = gcp['dados_centro'].apply(lambda x: x['id'] if x is not None else None)
gcp.loc[:, 'dados_centro_descricao'] = gcp['dados_centro'].apply(lambda x: x['descricao'] if x is not None else None)
gcp.loc[:, 'tipo_gn'] = gcp['dados_co_tipo_gn'].apply(lambda x: x['ds_descricao'] if x is not None else None)

gcp_modelado = gcp.drop(['dados_centro', 'dados_centro_id', 'dados_co_tipo_gn'], axis=1)
gcp_modelado.head(5)

,dt_ultimo_certificado_gcp,ds_nome,dados_centro_descricao,tipo_gn
0,None,Emerson Aredes,None,Suporte TI
1,None,Kleber Giungi,None,Equipe padrão (adicionada automaticamente para...
2,None,Suporte Polo Trial,None,Suporte
3,2020-09-28,Dra. Caroline Cândida Carvalho de Oliveira,Leforte HMCG,Pesquisador Principal
4,None,Dra. Tânia Caltabiano,Leforte HMCG,Pesquisador Principal


In [5]:
def verificar_vencimento_contratos(df_contratos, nome_centro, dias_para_vencimento=30):

    df_contratos['Assinatura do GCP'] = pd.to_datetime(df_contratos['Assinatura do GCP'], errors='coerce')

    hoje = datetime.today()
    limite_vencimento = hoje + timedelta(days=dias_para_vencimento)

    df_contratos['data_vencimento'] = df_contratos['Assinatura do GCP'] + timedelta(days=2 * 365)

    filtro_vencendo = (df_contratos['data_vencimento'] > hoje) & (df_contratos['data_vencimento'] <= limite_vencimento)

    filtro_sem_data = df_contratos['Assinatura do GCP'].isna()

    contratos_relevantes = df_contratos[filtro_vencendo | filtro_sem_data]

    if contratos_relevantes.empty:
        print(f"🔹 Não há contratos GCP sem data registrada no sistema ou com vencimento nos próximos {dias_para_vencimento} dias para {nome_centro}.")
    else:
        print(f"⚠️ Contratos GCP sem data registrada no sistema ou com vencimento nos próximos {dias_para_vencimento} dias para {nome_centro}:")
        print(contratos_relevantes[['Centro', 'Nome', 'Assinatura do GCP', 'data_vencimento']])

    return contratos_relevantes

In [6]:
hmcg = ['Leforte HMCG','Leforte Morumbi','Hospital Municipal de Barueri', 'Leforte Liberdade', 'Clinica CardialMed']
filtro1 = gcp_modelado['dados_centro_descricao'].isin(hmcg)
centro_hmcg = gcp_modelado[filtro1]
venc_gcp_hmcg = centro_hmcg[centro_hmcg['tipo_gn'] != 'Equipe padrão (adicionada automaticamente para todos os protocolos)']
venc_gcp_hmcg.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)
nova_ordem = ['Centro', 'Nome', 'Função', 'Assinatura do GCP']
venc_gcp_hmcg = venc_gcp_hmcg[nova_ordem] 
venc_gcp_hmcg

C:\Users\Gleice Barros\AppData\Local\Temp\ipykernel_18672\559468711.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venc_gcp_hmcg.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)


,Centro,Nome,Função,Assinatura do GCP
3,Leforte HMCG,Dra. Caroline Cândida Carvalho de Oliveira,Pesquisador Principal,2020-09-28
4,Leforte HMCG,Dra. Tânia Caltabiano,Pesquisador Principal,None
5,Leforte HMCG,Dra. Tânia Navarro,Pesquisador Principal,None
6,Leforte HMCG,Dra. Tassia Mancillha,Subinvestigador,None
14,Leforte HMCG,Isabel Santos,Coordenador de Estudo,None
...,...,...,...,...
3317,Leforte HMCG,Dr. EduardoTieppo,Equipe Médica,2024-02-20
3385,Leforte Liberdade,Dra. Marcela Costa,Pesquisador Principal,None
3392,Leforte Liberdade,Dr. Murilo Bispo dos Reis,Pesquisador Principal,None
3396,Leforte Liberdade,Dr. Nicolau Moreira Abrahão,Pesquisador Principal,None


In [7]:
rocio = [' Maternidade e Cirurgia Nossa Senhora do Rocio SA']
filtro2 = gcp_modelado['dados_centro_descricao'].isin(rocio)
centro_rocio = gcp_modelado[filtro2]
venc_gcp_rocio = centro_rocio [centro_rocio['tipo_gn'] != 'Equipe padrão (adicionada automaticamente para todos os protocolos)']
venc_gcp_rocio.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)
nova_ordem = ['Centro', 'Nome', 'Função', 'Assinatura do GCP']
venc_gcp_rocio = venc_gcp_rocio[nova_ordem] 
venc_gcp_rocio

,Centro,Nome,Função,Assinatura do GCP


In [8]:
iir_coord = ['Hospital das Clínicas de Itajubá', 'Saint-Beaute Clinique', 'Hospital Salvalus','Consultório Lopes e Sartorelli', ' Clínica Berger', 
            'QualiVida Higienópolis','Endolap Saúde' ]
filtro3 = gcp_modelado['dados_centro_descricao'].isin(iir_coord)
centro_iir_coord = gcp_modelado[filtro3]
venc_gcp_iir_coord = centro_iir_coord [centro_iir_coord['tipo_gn'] != 'Equipe padrão (adicionada automaticamente para todos os protocolos)']
venc_gcp_iir_coord.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)
nova_ordem = ['Centro', 'Nome', 'Função', 'Assinatura do GCP']
venc_gcp_iir_coord = venc_gcp_iir_coord[nova_ordem] 
venc_gcp_iir_coord

C:\Users\Gleice Barros\AppData\Local\Temp\ipykernel_18672\2634511288.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venc_gcp_iir_coord.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)


,Centro,Nome,Função,Assinatura do GCP
2024,Hospital Salvalus,Eduardo Augusto Rabelo Socca,Biologista,None
2267,QualiVida Higienópolis,Dra. Renata Tortato Meneguetti,Pesquisador Principal,None
2323,Endolap Saúde,Dra. Eloá Spritze Guollo,Pesquisador Principal,None
2324,Hospital Salvalus,Dra. Luciana Crema,Pesquisador Principal,None
2393,QualiVida Higienópolis,Dra. Eliana Araujo da Silva,Pesquisador Principal,None
2424,Hospital Salvalus,Raul Alberto Valiente,Pesquisador Principal,None
2598,QualiVida Higienópolis,Dra. Débora Cordeiro do Rosário,Pesquisador Principal,None
2607,Hospital Salvalus,Carina Carraro,Coordenador de Estudo,None
2608,Hospital Salvalus,Tainara Gramalio,Coordenador de Estudo,None
2655,QualiVida Higienópolis,Dra. Renée Mignolo Tanaka Ferreira,Pesquisador Principal,None


In [9]:
envio_viviane = ['Hospital Pilar','Santa casa de São Paulo','Hospital São Francisco de Ribeirã Preto',
                'Hospital Antônio Prudente','Clínica CardialMed','CLINAR - Clínica de Aparelhos RespiratÃ³rios',
                'Santa Casa de Fortaleza','Hospital Vera Cruz','Hapvida','Unimed Brusque',
                'Hospital São José das Doenças Infecciosas','Otorhinus Clínica Médica','Hospital São Francisco de Araraquara',
                'Maternidade Octaviano Neves','Clínica Infectologie','Hospital Teresa de Lisieux',
                'Hospital RioMar de Belém','Hospital e Maternidade Eugênia Pinheiro',
                'Hospital São Francisco Saúde','Hospital do Coração de Campinas','Aliança Cavernoma Brasil',
                'Faculdade de Medicina de Ribeirão Preto - USP ','Centro Clínico Zona Sul ']
filtro4 = gcp_modelado['dados_centro_descricao'].isin(envio_viviane)
centro_envio_viviane = gcp_modelado[filtro4]
venc_gcp_envio_viviane = centro_envio_viviane[centro_envio_viviane['tipo_gn'] != 'Equipe padrão (adicionada automaticamente para todos os protocolos)']
venc_gcp_envio_viviane.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)
nova_ordem = ['Centro', 'Nome', 'Função', 'Assinatura do GCP']
venc_gcp_envio_viviane = venc_gcp_envio_viviane[nova_ordem] 

venc_gcp_envio_viviane

C:\Users\Gleice Barros\AppData\Local\Temp\ipykernel_18672\610327026.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venc_gcp_envio_viviane.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)


,Centro,Nome,Função,Assinatura do GCP
50,Santa casa de São Paulo,Dr. Giuliano Giova Volpiani,Pesquisador Principal,None
51,Santa casa de São Paulo,Dr. Ronaldo Rabello,Pesquisador Principal,None
52,Santa casa de São Paulo,Dr. Rubens José Gagliardi,Pesquisador Principal,None
53,Santa casa de São Paulo,Dra. Walkiria Hueb Bernardi,Pesquisador Principal,2020-05-13
54,Clínica CardialMed,Dr. Heron Rhydan Saad Rached,Pesquisador Principal,None
66,Santa casa de São Paulo,Dr. Renato Jorge Alves,Pesquisador Principal,2019-03-06
73,Hospital Antônio Prudente,Dr. Bruno Cavalcante,Pesquisador Principal,None
74,Santa Casa de Fortaleza,Luiz Antonio Noleto Guimaraes,Subinvestigador,None
1771,Hapvida,pi.hapvida@svriglobal.com,Pesquisador Principal,None
1810,Clínica Infectologie,Dra. Raquel Bandeira da Silva,Pesquisador Principal,None


In [10]:
stacasa_santos = ['Santa Casa de Santos' ]
filtro5 = gcp_modelado['dados_centro_descricao'].isin(stacasa_santos)
centro_stacasa_santos = gcp_modelado[filtro5]
venc_gcp_stacasa_santos = centro_stacasa_santos [centro_stacasa_santos['tipo_gn'] != 'Equipe padrão (adicionada automaticamente para todos os protocolos)']
venc_gcp_stacasa_santos.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)
nova_ordem = ['Centro', 'Nome', 'Função', 'Assinatura do GCP']
venc_gcp_stacasa_santos = venc_gcp_stacasa_santos[nova_ordem]
venc_gcp_stacasa_santos

C:\Users\Gleice Barros\AppData\Local\Temp\ipykernel_18672\3690325233.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venc_gcp_stacasa_santos.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)


,Centro,Nome,Função,Assinatura do GCP
60,Santa Casa de Santos,Dr. Franklein Vieira Maia,Pesquisador Principal,None
70,Santa Casa de Santos,Dr. Marcelo Pilnik,Pesquisador Principal,None
80,Santa Casa de Santos,Dr. André Sementilli,Pesquisador Principal,None
1655,Santa Casa de Santos,Elaine Maria Borges Mancilha,Pesquisador Principal,None
1660,Santa Casa de Santos,Dr. José Eduardo Fernandes Vasconcelos,Pesquisador Principal,None
...,...,...,...,...
3313,Santa Casa de Santos,Dra. Carolina Yamaguti Chaud,Subinvestigador,None
3327,Santa Casa de Santos,Dra Magda Prates,Subinvestigador,None
3335,Santa Casa de Santos,Dra. Eva Ines Arabe Pereira,Subinvestigador,None
3343,Santa Casa de Santos,Dr. Thiago Fernandes Leomil,Subinvestigador,None


In [11]:
capibaribe = ['Hospital do Capibaribe']
filtro6 = gcp_modelado['dados_centro_descricao'].isin(capibaribe)
centro_capibaribe = gcp_modelado[filtro6]
venc_gcp_capibaribe = centro_capibaribe [centro_capibaribe['tipo_gn'] != 'Equipe padrão (adicionada automaticamente para todos os protocolos)']
venc_gcp_capibaribe.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)
nova_ordem = ['Centro', 'Nome', 'Função', 'Assinatura do GCP']
venc_gcp_capibaribe = venc_gcp_capibaribe[nova_ordem]
venc_gcp_capibaribe

C:\Users\Gleice Barros\AppData\Local\Temp\ipykernel_18672\635713975.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venc_gcp_capibaribe.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função'}, inplace=True)


,Centro,Nome,Função,Assinatura do GCP
2369,Hospital do Capibaribe,Mariane Teodoro Fernandes Rodrigues,Pesquisador Principal,None
2481,Hospital do Capibaribe,Leandro Apolinário da Silva,Subinvestigador,None
2589,Hospital do Capibaribe,Thiago Melo Rodrigues de Mariz,Subinvestigador,None
2590,Hospital do Capibaribe,Ana Dulce Lima de Luna Freire,Subinvestigador,None
2591,Hospital do Capibaribe,Icaro Matheus Felix Santos,Subinvestigador,None
2592,Hospital do Capibaribe,Carolina Natércia da Silva Lira,Subinvestigador,None
2593,Hospital do Capibaribe,Elton Menezes Gomes Silva,Subinvestigador,None
2776,Hospital do Capibaribe,Dr. Arlon Breno Figueiredo Nunes da Silveira,Equipe Médica,2023-10-30
3231,Hospital do Capibaribe,Ana Beatriz Evangelista,Estagiário/Assistente de Coordenação,2024-08-05
3232,Hospital do Capibaribe,Rúbia Dias Carvalho,Farmacêutico,None


In [12]:
contratos_hmcg = verificar_vencimento_contratos(venc_gcp_hmcg, "HMCG", dias_para_vencimento=30)
contratos_hmcg

⚠️ Contratos GCP sem data registrada no sistema ou com vencimento nos próximos 30 dias para HMCG:
                 Centro                           Nome Assinatura do GCP  \
4          Leforte HMCG          Dra. Tânia Caltabiano               NaT   
5          Leforte HMCG             Dra. Tânia Navarro               NaT   
6          Leforte HMCG          Dra. Tassia Mancillha               NaT   
14         Leforte HMCG                  Isabel Santos               NaT   
15         Leforte HMCG                  Giovana Rosas               NaT   
...                 ...                            ...               ...   
3299       Leforte HMCG       Dr. Marcelo Silva Soares               NaT   
3385  Leforte Liberdade             Dra. Marcela Costa               NaT   
3392  Leforte Liberdade      Dr. Murilo Bispo dos Reis               NaT   
3396  Leforte Liberdade    Dr. Nicolau Moreira Abrahão               NaT   
3401       Leforte HMCG  Dr. Leandro Tuzuki Cavalheiro            

,Centro,Nome,Função,Assinatura do GCP,data_vencimento
4,Leforte HMCG,Dra. Tânia Caltabiano,Pesquisador Principal,NaT,NaT
5,Leforte HMCG,Dra. Tânia Navarro,Pesquisador Principal,NaT,NaT
6,Leforte HMCG,Dra. Tassia Mancillha,Subinvestigador,NaT,NaT
14,Leforte HMCG,Isabel Santos,Coordenador de Estudo,NaT,NaT
15,Leforte HMCG,Giovana Rosas,None,NaT,NaT
...,...,...,...,...,...
3299,Leforte HMCG,Dr. Marcelo Silva Soares,Pesquisador Principal,NaT,NaT
3385,Leforte Liberdade,Dra. Marcela Costa,Pesquisador Principal,NaT,NaT
3392,Leforte Liberdade,Dr. Murilo Bispo dos Reis,Pesquisador Principal,NaT,NaT
3396,Leforte Liberdade,Dr. Nicolau Moreira Abrahão,Pesquisador Principal,NaT,NaT


In [13]:
contratos_rocio = verificar_vencimento_contratos(venc_gcp_rocio, "Rocío", dias_para_vencimento=30)

🔹 Não há contratos GCP sem data registrada no sistema ou com vencimento nos próximos 30 dias para Rocío.


In [14]:
contratos_iir_coord = verificar_vencimento_contratos(venc_gcp_iir_coord, "IRR COORD", dias_para_vencimento=30)
contratos_iir_coord

⚠️ Contratos GCP sem data registrada no sistema ou com vencimento nos próximos 30 dias para IRR COORD:
                              Centro  \
2024               Hospital Salvalus   
2267          QualiVida Higienópolis   
2323                   Endolap Saúde   
2324               Hospital Salvalus   
2393          QualiVida Higienópolis   
2424               Hospital Salvalus   
2598          QualiVida Higienópolis   
2607               Hospital Salvalus   
2608               Hospital Salvalus   
2655          QualiVida Higienópolis   
2663          QualiVida Higienópolis   
2696               Hospital Salvalus   
2751          QualiVida Higienópolis   
2833  Consultório Lopes e Sartorelli   
2843          QualiVida Higienópolis   
3022               Hospital Salvalus   
3050               Hospital Salvalus   
3081          QualiVida Higienópolis   
3138          QualiVida Higienópolis   
3166          QualiVida Higienópolis   
3286          QualiVida Higienópolis   
3386          Qua

,Centro,Nome,Função,Assinatura do GCP,data_vencimento
2024,Hospital Salvalus,Eduardo Augusto Rabelo Socca,Biologista,NaT,NaT
2267,QualiVida Higienópolis,Dra. Renata Tortato Meneguetti,Pesquisador Principal,NaT,NaT
2323,Endolap Saúde,Dra. Eloá Spritze Guollo,Pesquisador Principal,NaT,NaT
2324,Hospital Salvalus,Dra. Luciana Crema,Pesquisador Principal,NaT,NaT
2393,QualiVida Higienópolis,Dra. Eliana Araujo da Silva,Pesquisador Principal,NaT,NaT
2424,Hospital Salvalus,Raul Alberto Valiente,Pesquisador Principal,NaT,NaT
2598,QualiVida Higienópolis,Dra. Débora Cordeiro do Rosário,Pesquisador Principal,NaT,NaT
2607,Hospital Salvalus,Carina Carraro,Coordenador de Estudo,NaT,NaT
2608,Hospital Salvalus,Tainara Gramalio,Coordenador de Estudo,NaT,NaT
2655,QualiVida Higienópolis,Dra. Renée Mignolo Tanaka Ferreira,Pesquisador Principal,NaT,NaT


In [15]:
contratos_vivi = verificar_vencimento_contratos(venc_gcp_envio_viviane, "Envio Viviane", dias_para_vencimento=30)
contratos_vivi

⚠️ Contratos GCP sem data registrada no sistema ou com vencimento nos próximos 30 dias para Envio Viviane:
                                              Centro  \
50                           Santa casa de São Paulo   
51                           Santa casa de São Paulo   
52                           Santa casa de São Paulo   
54                                Clínica CardialMed   
73                         Hospital Antônio Prudente   
74                           Santa Casa de Fortaleza   
1771                                         Hapvida   
1810                            Clínica Infectologie   
1813                              Hospital Vera Cruz   
1822                              Hospital Vera Cruz   
1878                       Hospital Antônio Prudente   
1915                     Maternidade Octaviano Neves   
1916                     Maternidade Octaviano Neves   
1917                      Hospital Teresa de Lisieux   
1928                    Hospital São Francisco Saúde 

,Centro,Nome,Função,Assinatura do GCP,data_vencimento
50,Santa casa de São Paulo,Dr. Giuliano Giova Volpiani,Pesquisador Principal,NaT,NaT
51,Santa casa de São Paulo,Dr. Ronaldo Rabello,Pesquisador Principal,NaT,NaT
52,Santa casa de São Paulo,Dr. Rubens José Gagliardi,Pesquisador Principal,NaT,NaT
54,Clínica CardialMed,Dr. Heron Rhydan Saad Rached,Pesquisador Principal,NaT,NaT
73,Hospital Antônio Prudente,Dr. Bruno Cavalcante,Pesquisador Principal,NaT,NaT
74,Santa Casa de Fortaleza,Luiz Antonio Noleto Guimaraes,Subinvestigador,NaT,NaT
1771,Hapvida,pi.hapvida@svriglobal.com,Pesquisador Principal,NaT,NaT
1810,Clínica Infectologie,Dra. Raquel Bandeira da Silva,Pesquisador Principal,NaT,NaT
1813,Hospital Vera Cruz,Dra. Deborah Estrella,Pesquisador Principal,NaT,NaT
1822,Hospital Vera Cruz,Dra. Camila Rios Bretas,Pesquisador Principal,NaT,NaT


In [16]:
contratos_stacsantos = verificar_vencimento_contratos(venc_gcp_stacasa_santos, "Santa Casa de Santos", dias_para_vencimento=30)
contratos_stacsantos

⚠️ Contratos GCP sem data registrada no sistema ou com vencimento nos próximos 30 dias para Santa Casa de Santos:
                    Centro                                    Nome  \
60    Santa Casa de Santos               Dr. Franklein Vieira Maia   
70    Santa Casa de Santos                      Dr. Marcelo Pilnik   
80    Santa Casa de Santos                    Dr. André Sementilli   
1655  Santa Casa de Santos            Elaine Maria Borges Mancilha   
1660  Santa Casa de Santos  Dr. José Eduardo Fernandes Vasconcelos   
...                    ...                                     ...   
3313  Santa Casa de Santos            Dra. Carolina Yamaguti Chaud   
3327  Santa Casa de Santos                        Dra Magda Prates   
3335  Santa Casa de Santos             Dra. Eva Ines Arabe Pereira   
3343  Santa Casa de Santos             Dr. Thiago Fernandes Leomil   
3374  Santa Casa de Santos                        Maria Ligia Lyra   

     Assinatura do GCP data_vencimento  
60  

,Centro,Nome,Função,Assinatura do GCP,data_vencimento
60,Santa Casa de Santos,Dr. Franklein Vieira Maia,Pesquisador Principal,NaT,NaT
70,Santa Casa de Santos,Dr. Marcelo Pilnik,Pesquisador Principal,NaT,NaT
80,Santa Casa de Santos,Dr. André Sementilli,Pesquisador Principal,NaT,NaT
1655,Santa Casa de Santos,Elaine Maria Borges Mancilha,Pesquisador Principal,NaT,NaT
1660,Santa Casa de Santos,Dr. José Eduardo Fernandes Vasconcelos,Pesquisador Principal,NaT,NaT
...,...,...,...,...,...
3313,Santa Casa de Santos,Dra. Carolina Yamaguti Chaud,Subinvestigador,NaT,NaT
3327,Santa Casa de Santos,Dra Magda Prates,Subinvestigador,NaT,NaT
3335,Santa Casa de Santos,Dra. Eva Ines Arabe Pereira,Subinvestigador,NaT,NaT
3343,Santa Casa de Santos,Dr. Thiago Fernandes Leomil,Subinvestigador,NaT,NaT


In [17]:
contratos_capiberibe = verificar_vencimento_contratos(venc_gcp_capibaribe, "Capiberibe", dias_para_vencimento=30)
contratos_capiberibe

⚠️ Contratos GCP sem data registrada no sistema ou com vencimento nos próximos 30 dias para Capiberibe:
                      Centro                                 Nome  \
2369  Hospital do Capibaribe  Mariane Teodoro Fernandes Rodrigues   
2481  Hospital do Capibaribe          Leandro Apolinário da Silva   
2589  Hospital do Capibaribe       Thiago Melo Rodrigues de Mariz   
2590  Hospital do Capibaribe        Ana Dulce Lima de Luna Freire   
2591  Hospital do Capibaribe           Icaro Matheus Felix Santos   
2592  Hospital do Capibaribe      Carolina Natércia da Silva Lira   
2593  Hospital do Capibaribe            Elton Menezes Gomes Silva   
3232  Hospital do Capibaribe                  Rúbia Dias Carvalho   

     Assinatura do GCP data_vencimento  
2369               NaT             NaT  
2481               NaT             NaT  
2589               NaT             NaT  
2590               NaT             NaT  
2591               NaT             NaT  
2592               NaT      

,Centro,Nome,Função,Assinatura do GCP,data_vencimento
2369,Hospital do Capibaribe,Mariane Teodoro Fernandes Rodrigues,Pesquisador Principal,NaT,NaT
2481,Hospital do Capibaribe,Leandro Apolinário da Silva,Subinvestigador,NaT,NaT
2589,Hospital do Capibaribe,Thiago Melo Rodrigues de Mariz,Subinvestigador,NaT,NaT
2590,Hospital do Capibaribe,Ana Dulce Lima de Luna Freire,Subinvestigador,NaT,NaT
2591,Hospital do Capibaribe,Icaro Matheus Felix Santos,Subinvestigador,NaT,NaT
2592,Hospital do Capibaribe,Carolina Natércia da Silva Lira,Subinvestigador,NaT,NaT
2593,Hospital do Capibaribe,Elton Menezes Gomes Silva,Subinvestigador,NaT,NaT
3232,Hospital do Capibaribe,Rúbia Dias Carvalho,Farmacêutico,NaT,NaT


In [21]:
css_hover = "<style>table {border-collapse: collapse; width: 100%;} th, td {border: 1px solid black; padding: 8px; text-align: left;} th {background-color: #007bff;color: white;} tr:hover {background-color: #fff59d; box-shadow: inset 0 0 10px rgba(255, 235, 59, 0.5);}</style>"

In [22]:
smtp_server = os.getenv("EMAIL_SERVER")
email_port = int(os.getenv("EMAIL_PORT"))
email_usuario = os.getenv("EMAIL_USERNAME")
email_senha = os.getenv("EMAIL_PASSWORD")
enviar_para = os.getenv('ENVIAR_PARA')
destinatario_hmcg = os.getenv('DESTINATARIO_HMCG')
destinatario_rocio =  os.getenv('DESTINATARIO_ROCIO')
destinatario_scs = os.getenv('DESTINATARIO_SCS')
destinatario_iir =  os.getenv('DESTINATARIO_IIR')
destinatario_vivi =  os.getenv('DESTINATARIO_VIVIANE')
destinatario_capiberibe =  os.getenv('DESTINATARIO_CAPIBERIBE')

In [20]:
def enviar_email(destinatário,df_tratado):
    try:
        if df_tratado is None or df_tratado.empty:
            print("Não há contratos vencidos para enviar por e-mail.")
            return

        tabela_html = df_tratado[['Centro', 'Nome', 'Assinatura do GCP', 'data_vencimento']].to_html(
            index=False, escape=False, justify="left", border=0, classes="table"
        )

        msg = MIMEMultipart("alternative")
        msg['From'] = email_usuario
        msg['To'] = email_usuario
        msg['Bcc'] = ', '.join(destinatário)
        msg['Subject'] = "GCP com vencimento nos próximos 30 dias ou sem data informada - "

        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <h2>Vencimento de GCP</h2>
                <p>Olá,</p> 
                <br>
                <p>Segue abaixo GCP que vencerão no próximo mês ou sem data informada no sistema:</p>
                {tabela_html}
                <p>Este email é gerado automaticamente a partir de informações inseridadas na Polo Trial.</p>
                <p>Qualquer dúvida, por favor, contate o <strong><span style="text-decoration: underline;">time BI - SVRI</span></strong>.</p>
            </body>
        </html>
        """
        msg.attach(MIMEText(body, 'html'))

        with smtplib.SMTP(smtp_server, email_port
    ) as server:
            server.starttls()
            server.login(email_usuario, email_senha)
            server.send_message(msg)

        print("E-mail de GCP que já venceram enviado com sucesso!")
    
    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

In [ ]:
enviar_email([destinatario_hmcg], contratos_hmcg)
enviar_email([destinatario_rocio], contratos_rocio)
enviar_email([destinatario_iir], contratos_iir_coord)
enviar_email([destinatario_scs], contratos_stacsantos)
enviar_email([destinatario_vivi], contratos_vivi)
enviar_email([destinatario_capiberibe], contratos_capiberibe)

E-mail de GCP que já venceram enviado com sucesso!
Não há contratos vencidos para enviar por e-mail.
E-mail de GCP que já venceram enviado com sucesso!
E-mail de GCP que já venceram enviado com sucesso!


# VISITAS DE SEGUIMENTO (MODELAGEM E FUNÇÕES DE ENVIO)

In [31]:
protocolo_url = urljoin(api_url, "/protocolo?nested=true")
headers = {"Authorization": auth_token}

response = requests.get(protocolo_url, headers=headers)
protocolo = response.json()
protocolo = pd.DataFrame(protocolo)
protocolo.head(2)

,id,titulo_protocolo,cor_agenda,numero_protocolo,apelido_protocolo,coordenador,pi,pesquisador_backup,co_pessoa_regulatorio,data_cadastro,...,dados_aprovacao_cep,dados_co_centro,PessoaPI,PessoaCoordenador,PessoaPesquisador,PessoaRegulatorio,fase_pesquisa,status,tipo_iniciativa,nome_patrocinador
0,4,"ENSAIO CLÍNICO FASE 2, RANDOMIZADO, DUPLO-CEGO...",#330eb9,BTK-COV-202BR,BTK,261.0,306.0,210.0,214.0,2021-04-05T18:37:50.000Z,...,"{'id': 60, 'ds_descricao': 'Sim'}","{'id': 3, 'descricao': 'Leforte HMCG'}","{'id': 306, 'ds_nome': 'Dr. Carlos Augusto Qua...","{'id': 261, 'ds_nome': 'Giovana Rosas'}",None,"{'id': 214, 'ds_nome': 'Priscila do Prado Costa'}","{'id': 123, 'ds_descricao': 'Fase III'}","{'id': 296, 'ds_descricao': 'Concluído'}","{'id': 506, 'ds_descricao': 'Patrocinador'}","{'id': 723, 'ds_descricao': 'Sorrento Therapeu..."
1,5,"ENSAIO CLÍNICO FASE 2, RANDOMIZADO, DUPLO-CEGO...",#330eb9,BTK-COV-202BR,BTK,310.0,199.0,227.0,214.0,2021-07-07T12:28:33.000Z,...,"{'id': 60, 'ds_descricao': 'Sim'}","{'id': 2, 'descricao': 'Leforte Liberdade'}","{'id': 199, 'ds_nome': 'Dra. Caroline Cândida ...","{'id': 310, 'ds_nome': 'Isabel Cristina dos Sa...","{'id': 227, 'ds_nome': 'Viviane Santana da Sil...","{'id': 214, 'ds_nome': 'Priscila do Prado Costa'}","{'id': 122, 'ds_descricao': 'Fase II'}","{'id': 296, 'ds_descricao': 'Concluído'}","{'id': 506, 'ds_descricao': 'Patrocinador'}","{'id': 723, 'ds_descricao': 'Sorrento Therapeu..."


In [32]:
centros = protocolo[['id','apelido_protocolo', 'numero_protocolo','co_externo','apelido_centro']].copy()
centros

,id,apelido_protocolo,numero_protocolo,co_externo,apelido_centro
0,4,BTK,BTK-COV-202BR,BTK-COV-202BR,BTK - Leforte HMCG
1,5,BTK,BTK-COV-202BR,BTK-COV-202BR,BTK - Leforte Liberdade
2,8,RIGEL-FOCUS,C-935788-061,C-935788-061,RIGEL-FOCUS - Leforte Liberdade
3,9,BIOTEST-998,ESsCOVID-998,None,BIOTEST-998 - Leforte HMCG
4,10,GARDEN,ACT-CS-006,ACT-CS-006,GARDEN - Leforte HMCG
...,...,...,...,...,...
2165,2376,EB ACTIVA,,None,EB ACTIVA - HELP – Hospital de Ensino e Labora...
2166,2377,EB ACTIVA,,None,EB ACTIVA - Leforte HMCG
2167,2378,Oxandrolona,,None,Oxandrolona - Hapvida
2168,2379,Oxandrolona,,None,Oxandrolona - UNAERP


In [33]:
participantes_visita_url = urljoin(api_url, "/participante_visita?nested=true")
headers = {"Authorization": auth_token}

response = requests.get(participantes_visita_url, headers=headers)
participantes_visita = response.json()
participantes_visita = pd.DataFrame(participantes_visita)

participantes_visita.head(2)

,id,co_participante,co_visita,status_participante_visita,nome_tarefa,data_estimada,data_estimada_fim,data_realizada,observacoes,valor_previsto,...,co_externo,data_preenchimento_visita,data_preenchimento_crf,agendada_usuario,dados_participante,dados_visita,dados_status,dados_nota_fiscal,dados_local,dados_responsavel
0,639,54,1.0,20,Triagem,2021-01-05T00:00:00.000Z,None,2021-01-05T00:00:00.000Z,None,7940.48,...,None,None,None,None,"{'id': 54, 'co_protocolo': 4, 'id_participante...","{'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...","{'id': 20, 'ds_descricao': 'Realizada'}","{'id': 784, 'codigo_nota_fiscal': 'NF 171'}",None,None
1,709,54,2.0,20,Rand + Dose 1 - D1,2021-01-06T00:00:00.000Z,None,2021-01-06T00:00:00.000Z,None,6484.08,...,None,None,None,None,"{'id': 54, 'co_protocolo': 4, 'id_participante...","{'id': 2, 'ds_nome_visita': 'Rand + Dose 1 - D...","{'id': 20, 'ds_descricao': 'Realizada'}","{'id': 784, 'codigo_nota_fiscal': 'NF 171'}",None,None


In [34]:
seguimento = participantes_visita[['dados_participante', 'data_estimada', 'dados_status', 'dados_visita']].copy()

seguimento.loc[:, 'dados_participante_id'] = seguimento['dados_participante'].apply(lambda x: x['id'] if x is not None else None)
seguimento.loc[:, 'id_participante'] = seguimento['dados_participante'].apply(lambda x: x['id_participante'] if x is not None else None)
seguimento.loc[:, 'dados_protocolo'] = seguimento['dados_participante'].apply(lambda x: x['dados_protocolo'] if x is not None else None)

seguimento.loc[:, 'dados_status_id'] = seguimento['dados_status'].apply(lambda x: x['id'] if x is not None else None)
seguimento.loc[:, 'ds_descricao'] = seguimento['dados_status'].apply(lambda x: x['ds_descricao'] if x is not None else None)
seguimento.loc[:, 'ds_nome_visita'] = seguimento['dados_visita'].apply(lambda x: x['ds_nome_visita'] if x is not None else None)

seguimento.loc[:, 'apelido_protocolo'] = seguimento['dados_protocolo'].apply(lambda x: x['apelido_protocolo'] if x is not None else None)
seguimento['data_estimada'] = pd.to_datetime(seguimento['data_estimada']).dt.date
seguimento.head(2)

,dados_participante,data_estimada,dados_status,dados_visita,dados_participante_id,id_participante,dados_protocolo,dados_status_id,ds_descricao,ds_nome_visita,apelido_protocolo
0,"{'id': 54, 'co_protocolo': 4, 'id_participante...",2021-01-05,"{'id': 20, 'ds_descricao': 'Realizada'}","{'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...",54,1001,"{'id': 4, 'apelido_protocolo': 'BTK'}",20,Realizada,Triagem,BTK
1,"{'id': 54, 'co_protocolo': 4, 'id_participante...",2021-01-06,"{'id': 20, 'ds_descricao': 'Realizada'}","{'id': 2, 'ds_nome_visita': 'Rand + Dose 1 - D...",54,1001,"{'id': 4, 'apelido_protocolo': 'BTK'}",20,Realizada,Rand + Dose 1 - D1,BTK


In [35]:
seguimento_tratado = seguimento.drop(['dados_participante','dados_status','id_participante','dados_status_id'], axis=1)
seguimento_tratado.loc[:, 'id'] = seguimento['dados_protocolo'].apply(lambda x: x['id'] if x is not None else None)
seguimento_tratado.head(2)

,data_estimada,dados_visita,dados_participante_id,dados_protocolo,ds_descricao,ds_nome_visita,apelido_protocolo,id
0,2021-01-05,"{'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...",54,"{'id': 4, 'apelido_protocolo': 'BTK'}",Realizada,Triagem,BTK,4
1,2021-01-06,"{'id': 2, 'ds_nome_visita': 'Rand + Dose 1 - D...",54,"{'id': 4, 'apelido_protocolo': 'BTK'}",Realizada,Rand + Dose 1 - D1,BTK,4


In [36]:
seguimentos = pd.merge(centros, seguimento_tratado, on='id', how='inner')
seguimentos

,id,apelido_protocolo_x,numero_protocolo,co_externo,apelido_centro,data_estimada,dados_visita,dados_participante_id,dados_protocolo,ds_descricao,ds_nome_visita,apelido_protocolo_y
0,4,BTK,BTK-COV-202BR,BTK-COV-202BR,BTK - Leforte HMCG,2021-01-05,"{'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...",54,"{'id': 4, 'apelido_protocolo': 'BTK'}",Realizada,Triagem,BTK
1,4,BTK,BTK-COV-202BR,BTK-COV-202BR,BTK - Leforte HMCG,2021-01-06,"{'id': 2, 'ds_nome_visita': 'Rand + Dose 1 - D...",54,"{'id': 4, 'apelido_protocolo': 'BTK'}",Realizada,Rand + Dose 1 - D1,BTK
2,4,BTK,BTK-COV-202BR,BTK-COV-202BR,BTK - Leforte HMCG,2021-01-07,"{'id': 7, 'ds_nome_visita': 'D2 - Hospitalizaç...",54,"{'id': 4, 'apelido_protocolo': 'BTK'}",Realizada,D2 - Hospitalização,BTK
3,4,BTK,BTK-COV-202BR,BTK-COV-202BR,BTK - Leforte HMCG,2021-01-08,"{'id': 8, 'ds_nome_visita': 'D3 - Hospitalizaç...",54,"{'id': 4, 'apelido_protocolo': 'BTK'}",Realizada,D3 - Hospitalização,BTK
4,4,BTK,BTK-COV-202BR,BTK-COV-202BR,BTK - Leforte HMCG,2021-01-09,"{'id': 9, 'ds_nome_visita': 'D4 - Hospitalizaç...",54,"{'id': 4, 'apelido_protocolo': 'BTK'}",Realizada,D4 - Hospitalização,BTK
...,...,...,...,...,...,...,...,...,...,...,...,...
15243,2057,Post Market Surveillance,SILIMED – INDÚSTRIA DE IMPLANTES LTDA,None,Post Market Surveillance - Centro Vila Olimpia,2025-03-07,"{'id': 2980, 'ds_nome_visita': 'Questionário S...",3008,"{'id': 2057, 'apelido_protocolo': 'Post Market...",Realizada,Questionário SILIMED,Post Market Surveillance
15244,2057,Post Market Surveillance,SILIMED – INDÚSTRIA DE IMPLANTES LTDA,None,Post Market Surveillance - Centro Vila Olimpia,2025-03-13,"{'id': 2980, 'ds_nome_visita': 'Questionário S...",3009,"{'id': 2057, 'apelido_protocolo': 'Post Market...",Realizada,Questionário SILIMED,Post Market Surveillance
15245,2057,Post Market Surveillance,SILIMED – INDÚSTRIA DE IMPLANTES LTDA,None,Post Market Surveillance - Centro Vila Olimpia,2025-03-13,"{'id': 2980, 'ds_nome_visita': 'Questionário S...",3010,"{'id': 2057, 'apelido_protocolo': 'Post Market...",Realizada,Questionário SILIMED,Post Market Surveillance
15246,2057,Post Market Surveillance,SILIMED – INDÚSTRIA DE IMPLANTES LTDA,None,Post Market Surveillance - Centro Vila Olimpia,2025-03-14,"{'id': 2980, 'ds_nome_visita': 'Questionário S...",3011,"{'id': 2057, 'apelido_protocolo': 'Post Market...",Pendente,Questionário SILIMED,Post Market Surveillance


In [37]:
seguimentos['apelido_protocolo_x'] = seguimentos['apelido_protocolo_x'].str.strip()
visitas_filtrado = seguimentos[seguimentos['ds_descricao'].str.contains('Pendente')]
nova_ordem = ["apelido_centro", "dados_participante_id", "ds_nome_visita","data_estimada","ds_descricao"]
visitas_reordenado = visitas_filtrado[nova_ordem]
visitas_reordenado.rename(columns={'apelido_centro':'Estudo/Centro', 'dados_participante_id': 'ID Participante', 'ds_nome_visita': 'Tipo Visita',
                          'data_estimada':'Data Estimada', 'ds_descricao':'Status'}, inplace=True)
visitas_reordenado

C:\Users\Gleice Barros\AppData\Local\Temp\ipykernel_3484\292463722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  visitas_reordenado.rename(columns={'apelido_centro':'Estudo/Centro', 'dados_participante_id': 'ID Participante', 'ds_nome_visita': 'Tipo Visita',


,Estudo/Centro,ID Participante,Tipo Visita,Data Estimada,Status
5338,VICTORION 2 - PREVENT - Leforte HMCG,2079,EOS,NaT,Pendente
5339,VICTORION 2 - PREVENT - Leforte HMCG,2079,Visita Unschedule,NaT,Pendente
5349,VICTORION 2 - PREVENT - Leforte HMCG,2079,Mês 33,2025-08-31,Pendente
5350,VICTORION 2 - PREVENT - Leforte HMCG,2079,Mês 39,2026-02-27,Pendente
5351,VICTORION 2 - PREVENT - Leforte HMCG,2079,Mês 45,2026-08-26,Pendente
...,...,...,...,...,...
15194,GLADE EXTENSÃO PK - Leforte Morumbi,2861,V13,NaT,Pendente
15195,GLADE EXTENSÃO PK - Leforte Morumbi,2861,V14,NaT,Pendente
15205,GLADE EXTENSÃO PK - Leforte Morumbi,2861,V10,2025-02-28,Pendente
15246,Post Market Surveillance - Centro Vila Olimpia,3011,Questionário SILIMED,2025-03-14,Pendente


In [38]:
datas_visitas = visitas_reordenado.dropna(subset=["Data Estimada"])
datas_visitas = datas_visitas.sort_values(by= "Data Estimada", ascending=True)
datas_visitas = datas_visitas[~datas_visitas["Tipo Visita"].isin(["Unscheduled", "Triagem", "End of Study"])]
valores_contagem = datas_visitas['Tipo Visita'].value_counts()
datas_visitas

,Estudo/Centro,ID Participante,Tipo Visita,Data Estimada,Status
11705,GLADE - Leforte Morumbi,2774,D90 - Visita 3,2024-10-06,Pendente
11706,GLADE - Leforte Morumbi,2774,D180 - Visita 4,2025-01-04,Pendente
14993,EXHALE-4 - Santa Casa de Santos,2955,Visita 2 (Semana 4),2025-02-12,Pendente
12727,EXHALE-2 - Santa Casa de Santos,2984,Triagem 2/Run in,2025-02-22,Pendente
11989,GLADE - Hospital de Clínicas de Itajubá,2979,D180 - Visita 4,2025-02-24,Pendente
...,...,...,...,...,...
11330,VICTORION-1-PREVENT - Leforte HMCG,2663,V14 - M75,2030-06-08,Pendente
11311,VICTORION-1-PREVENT - Leforte HMCG,2662,V14 - M75,2030-06-08,Pendente
11387,VICTORION-1-PREVENT - Leforte HMCG,2687,V14 - M75,2030-07-01,Pendente
11349,VICTORION-1-PREVENT - Leforte HMCG,2682,V14 - M75,2030-07-01,Pendente


In [39]:
def verificar_visitas_proximos_dias(datas_visitas, dias_para_visita=20):
    datas_visitas['Data Estimada'] = pd.to_datetime(datas_visitas['Data Estimada'])
    
    hoje = datetime.today()
    limite_visita = hoje + timedelta(days=dias_para_visita)
    
    visitas_futuras = datas_visitas[
        (datas_visitas['Data Estimada'] >= hoje) & 
        (datas_visitas['Data Estimada'] <= limite_visita)
    ]
    
    if visitas_futuras.empty:
        print(f"Não há visitas programadas para os próximos {dias_para_visita} dias.")
    else:
        print(f"Visitas programadas para os próximos {dias_para_visita} dias:")
        print(visitas_futuras[['Estudo/Centro', 'ID Participante', 'Tipo Visita', 'Data Estimada','Status']])
    
    return visitas_futuras

visitas_20_dias = verificar_visitas_proximos_dias(datas_visitas, dias_para_visita=20)
# próximas_visitas = visitas_20_dias.drop(['dados_visita'], axis=1)
# próximas_visitas.head(2)

Visitas programadas para os próximos 20 dias:
                                           Estudo/Centro  ID Participante  \
7050                       GALAXI - Santa Casa de Santos             2129   
14862  MK0616-015  - Maternidade e Cirurgia Nossa Sen...             2965   
14835  MK0616-015  - Maternidade e Cirurgia Nossa Sen...             2964   
14720  MK0616-015  - Maternidade e Cirurgia Nossa Sen...             2945   
14077  MK0616-015  - Maternidade e Cirurgia Nossa Sen...             2852   
6099                VICTORION 2 - PREVENT - Leforte HMCG             2189   
6657                VICTORION 2 - PREVENT - Leforte HMCG             2350   
10800                 VICTORION-1-PREVENT - Leforte HMCG             2513   
13493  MK0616-015  - Maternidade e Cirurgia Nossa Sen...             2747   
14020  MK0616-015  - Maternidade e Cirurgia Nossa Sen...             2842   
13380  MK0616-015  - Maternidade e Cirurgia Nossa Sen...             2738   
13334  MK0616-015  - Maternida

In [42]:
def enviar_emails(próximas_visitas):
    global enviar_para  # Acessa a variável global 'enviar_para'
    
    try:
        if próximas_visitas is None or próximas_visitas.empty:
            print("Não há visitas nos próximos dias para enviar por e-mail.")
            return

        colunas_esperadas = {'Estudo/Centro', 'ID Participante', 'Tipo Visita', 'Data Estimada', 'Status'}
        if not colunas_esperadas.issubset(próximas_visitas.columns):
            print("Erro: DataFrame não contém todas as colunas esperadas.")
            return

        if not all([smtp_server, email_usuario, email_senha, email_port]):
            print("Erro: Configurações de e-mail estão incompletas.")
            return

        # Validação de e-mails com regex
        email_regex = r'^[\w\.-]+@[\w\.-]+\.\w+$'
        if isinstance(enviar_para, list):
            enviar_para = [email.strip() for email in enviar_para if email.strip() and re.match(email_regex, email.strip())]
        else:
            enviar_para = []

        # Criação da tabela HTML
        tabela_html = próximas_visitas[list(colunas_esperadas)].to_html(
            index=False, escape=False, justify="left", border=0, classes="table"
        )

        # Montagem da mensagem
        msg = MIMEMultipart("alternative")
        msg['From'] = email_usuario
        msg['To'] = email_usuario
        msg['Subject'] = "Visitas de Seguimento - próximos 20 dias"
        
        # Define o campo BCC apenas se houver destinatários válidos
        if enviar_para:
            msg['Bcc'] = ', '.join(enviar_para)

        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <p>Olá,</p>
                <p>Segue abaixo lista com visitas de seguimento programadas para os próximos 20 dias.</p>
                {tabela_html}
                <p>Este email é gerado automaticamente a partir de informações inseridas na Polo Trial.</p>
                <p>Qualquer dúvida, por favor, contate o <strong><span style="text-decoration: underline;">time BI - SVRI</span></strong>.</p>
            </body>
        </html>
        """
        msg.attach(MIMEText(body, 'html'))

        # Envio de e-mail
        with smtplib.SMTP(smtp_server, email_port) as server:
            server.starttls()
            server.login(email_usuario, email_senha)
            
            # Lista final de destinatários
            destinatarios = [email_usuario] + enviar_para
            if destinatarios:
                server.sendmail(email_usuario, destinatarios, msg.as_string())
                print("E-mail de visitas pendentes enviado com sucesso!")
            else:
                print("Nenhum destinatário válido. E-mail não enviado.")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

# Chamada da função
enviar_emails(visitas_20_dias)

E-mail de visitas pendentes enviado com sucesso!


# VISITAS DE MONITORIA (MODELAGEM E FUNÇÃO ENVIO DE EMAIL)

In [23]:
agenda_url = urljoin(api_url, "/agenda?nested=true")
headers = {"Authorization": auth_token}

response = requests.get(agenda_url, headers=headers)
agenda = response.json()
agenda = pd.DataFrame(agenda)

agenda.head(2)

,id,tipo_evento,titulo,ds_co_industria,ds_co_hc,co_participante,tipo,status_evento,data_estimada,data_estimada_fim,...,data_agenda,color,title,co_visita_referencia,filtro_dias,co_externo,dados_tipo,dados_status,dados_protocolo,dados_tipo_evento
0,1,1,Centriguga,,,,4,20,2022-04-28T00:00:00.000Z,None,...,2023-08-23T00:00:00.000Z,#428F54,Centriguga,None,1056.0,None,"{'id': 4, 'tipo': 'Calibração'}","{'id': 20, 'ds_descricao': 'Realizada'}",None,"{'id': 1, 'descricao': 'Evento'}"
1,4,1,contrato V6,,,,6,20,2022-07-06T00:00:00.000Z,None,...,2022-07-06T00:00:00.000Z,#000000,contrato V6,None,987.0,None,"{'id': 6, 'tipo': 'Contrato'}","{'id': 20, 'ds_descricao': 'Realizada'}","{'id': 53, 'apelido_protocolo': 'Dipro'}","{'id': 1, 'descricao': 'Evento'}"


In [24]:
monitoria = agenda[['dados_protocolo', 'dados_tipo', 'dados_status', 'data_estimada_filter']].copy()

monitoria.loc[:, 'dados_protocolo_id'] = monitoria['dados_protocolo'].apply(lambda x: x['id'] if x is not None else None)
monitoria.loc[:, 'apelido_protocolo'] = monitoria['dados_protocolo'].apply(lambda x: x['apelido_protocolo'] if x is not None else None)

monitoria.loc[:, 'dados_tipo_id'] = monitoria['dados_tipo'].apply(lambda x: x['id'] if x is not None else None)
monitoria.loc[:, 'dados_tipo_tipo'] = monitoria['dados_tipo'].apply(lambda x: x['tipo'] if x is not None else None)

monitoria.loc[:, 'dados_status_id'] = monitoria['dados_status'].apply(lambda x: x['id'] if x is not None else None)
monitoria.loc[:, 'ds_descricao'] = monitoria['dados_status'].apply(lambda x: x['ds_descricao'] if x is not None else None)

monitoria = monitoria.drop(['dados_protocolo','dados_status', 'dados_tipo_id','dados_status_id'], axis=1)
monitoria.rename(columns={'dados_protocolo_id': 'id'}, inplace=True)

monitoria.head(2)

,dados_tipo,data_estimada_filter,id,apelido_protocolo,dados_tipo_tipo,ds_descricao
0,"{'id': 4, 'tipo': 'Calibração'}",2022-04-28,NaN,None,Calibração,Realizada
1,"{'id': 6, 'tipo': 'Contrato'}",2022-07-06,53.0,Dipro,Contrato,Realizada


In [25]:
monitorias = pd.merge(centros, monitoria, on='id', how='inner')
monitorias

,id,apelido_protocolo_x,numero_protocolo,co_externo,apelido_centro,dados_tipo,data_estimada_filter,apelido_protocolo_y,dados_tipo_tipo,ds_descricao
0,4,BTK,BTK-COV-202BR,BTK-COV-202BR,BTK - Leforte HMCG,"{'id': 1, 'tipo': 'Visita'}",2021-01-05,BTK,Visita,Realizada
1,4,BTK,BTK-COV-202BR,BTK-COV-202BR,BTK - Leforte HMCG,"{'id': 1, 'tipo': 'Visita'}",2021-01-07,BTK,Visita,Realizada
2,4,BTK,BTK-COV-202BR,BTK-COV-202BR,BTK - Leforte HMCG,"{'id': 1, 'tipo': 'Visita'}",2021-01-08,BTK,Visita,Realizada
3,4,BTK,BTK-COV-202BR,BTK-COV-202BR,BTK - Leforte HMCG,"{'id': 1, 'tipo': 'Visita'}",2021-01-12,BTK,Visita,Realizada
4,4,BTK,BTK-COV-202BR,BTK-COV-202BR,BTK - Leforte HMCG,"{'id': 1, 'tipo': 'Visita'}",2021-01-14,BTK,Visita,Realizada
...,...,...,...,...,...,...,...,...,...,...
15459,2057,Post Market Surveillance,SILIMED – INDÚSTRIA DE IMPLANTES LTDA,None,Post Market Surveillance - Centro Vila Olimpia,"{'id': 1, 'tipo': 'Visita'}",2025-03-07,Post Market Surveillance,Visita,Realizada
15460,2057,Post Market Surveillance,SILIMED – INDÚSTRIA DE IMPLANTES LTDA,None,Post Market Surveillance - Centro Vila Olimpia,"{'id': 1, 'tipo': 'Visita'}",2025-03-13,Post Market Surveillance,Visita,Realizada
15461,2057,Post Market Surveillance,SILIMED – INDÚSTRIA DE IMPLANTES LTDA,None,Post Market Surveillance - Centro Vila Olimpia,"{'id': 1, 'tipo': 'Visita'}",2025-03-13,Post Market Surveillance,Visita,Realizada
15462,2057,Post Market Surveillance,SILIMED – INDÚSTRIA DE IMPLANTES LTDA,None,Post Market Surveillance - Centro Vila Olimpia,"{'id': 1, 'tipo': 'Visita'}",2025-03-14,Post Market Surveillance,Visita,Pendente


In [26]:
monitorias_filtrado = monitorias[monitorias['dados_tipo_tipo'].str.contains('Monitoria')]
monitorias_filtrado['apelido_protocolo_x'] = monitorias_filtrado['apelido_protocolo_x'].str.strip()

monitorias_filtrado

C:\Users\Gleice Barros\AppData\Local\Temp\ipykernel_3484\2407812721.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monitorias_filtrado['apelido_protocolo_x'] = monitorias_filtrado['apelido_protocolo_x'].str.strip()


,id,apelido_protocolo_x,numero_protocolo,co_externo,apelido_centro,dados_tipo,data_estimada_filter,apelido_protocolo_y,dados_tipo_tipo,ds_descricao
663,11,DEFLECT,MPZ-II-02,MPZ-II-02,DEFLECT - Maternidade e Cirurgia Nossa Senhora...,"{'id': 2, 'tipo': 'Monitoria'}",2023-05-23,DEFLECT,Monitoria,Realizada
2374,32,AEGIS,CSL112_3001,CSL112_3001,AEGIS - Leforte HMCG,"{'id': 2, 'tipo': 'Monitoria'}",2023-06-26,AEGIS,Monitoria,Realizada
2375,32,AEGIS,CSL112_3001,CSL112_3001,AEGIS - Leforte HMCG,"{'id': 2, 'tipo': 'Monitoria'}",2023-07-26,AEGIS,Monitoria,Realizada
2376,32,AEGIS,CSL112_3001,CSL112_3001,AEGIS - Leforte HMCG,"{'id': 2, 'tipo': 'Monitoria'}",2024-01-15,AEGIS,Monitoria,Realizada
2721,34,DIA3018,JNJ-28431754,DIA,DIA3018 - Leforte HMCG,"{'id': 2, 'tipo': 'Monitoria'}",2023-09-05,DIA3018,Monitoria,Realizada
...,...,...,...,...,...,...,...,...,...,...
15177,1492,EXHALE-4,AR-DEX-22-03,None,EXHALE-4 - Santa Casa de Santos,"{'id': 2, 'tipo': 'Monitoria'}",2025-04-02,EXHALE-4,Monitoria,Pendente
15232,1493,ABTECT-107 - Maintenance,ABX464-107,None,ABTECT-107 - Maintenance - Santa Casa de Santos,"{'id': 2, 'tipo': 'Monitoria'}",2025-02-28,ABTECT-107 - Maintenance,Monitoria,Pendente
15236,1495,LATAM LOWERS LDL-C,CKJX839A1MX02,None,LATAM LOWERS LDL-C - Maternidade e Cirurgia No...,"{'id': 2, 'tipo': 'Monitoria'}",2025-04-11,LATAM LOWERS LDL-C,Monitoria,Pendente
15449,1661,BIOTEST-996,BIOTEST-996,None,BIOTEST-996 - Maternidade e Cirurgia Nossa Sen...,"{'id': 2, 'tipo': 'Monitoria'}",2024-07-03,BIOTEST-996,Monitoria,Realizada


In [27]:
def verificar_monitorias_pendentes():
    monitorias_filtrado.loc[:, 'data_estimada_filter'] = pd.to_datetime(monitorias_filtrado['data_estimada_filter'])
    hoje = datetime.today()

    monitorias_pendentes = monitorias_filtrado[
        (monitorias_filtrado['ds_descricao'] == 'Pendente') &
        (monitorias_filtrado['data_estimada_filter'] >= hoje)
    ]
    monitorias_pendentes = monitorias_pendentes.sort_values(by='data_estimada_filter', ascending=True)

    print(monitorias_pendentes[['dados_tipo_tipo', 'data_estimada_filter', 'apelido_protocolo_x', 'ds_descricao','apelido_centro']])
    return monitorias_pendentes

schedule.every().monday.at("09:00").do(verificar_monitorias_pendentes)

monitorias_pendentes_df = verificar_monitorias_pendentes()

      dados_tipo_tipo data_estimada_filter       apelido_protocolo_x  \
7174        Monitoria  2025-03-21 00:00:00           LIBREXIA-STROKE   
10326       Monitoria  2025-03-24 00:00:00  BIOTEST- Tricovid - 1001   
11694       Monitoria  2025-03-25 00:00:00                  EXHALE-2   
10247       Monitoria  2025-03-31 00:00:00  BIOTEST- Tricovid - 1001   
12195       Monitoria  2025-03-31 00:00:00                    GLORIA   
7173        Monitoria  2025-04-02 00:00:00           LIBREXIA-STROKE   
15176       Monitoria  2025-04-02 00:00:00                  EXHALE-4   
15177       Monitoria  2025-04-02 00:00:00                  EXHALE-4   
5364        Monitoria  2025-04-04 00:00:00     VICTORION 2 - PREVENT   
12246       Monitoria  2025-04-08 00:00:00                   FREXALT   
11511       Monitoria  2025-04-10 00:00:00                    GLORIA   
15236       Monitoria  2025-04-11 00:00:00        LATAM LOWERS LDL-C   
12176       Monitoria  2025-05-13 00:00:00              TAK-330-

In [28]:
envio_para = os.getenv('ENVIO_PARA')

In [29]:
monitorias_pendentes_df.rename(columns={'apelido_centro':'Estudo/Centro', 'dados_tipo_tipo': 'Monitoria', 'data_estimada_filter': 'Data Estimada',
                                        'ds_descricao': 'Status'}, inplace=True)
monitorias_pendentes_df

,id,apelido_protocolo_x,numero_protocolo,co_externo,Estudo/Centro,dados_tipo,Data Estimada,apelido_protocolo_y,Monitoria,Status
7174,169,LIBREXIA-STROKE,70033093STR3001,STRUCK,LIBREXIA-STROKE - Maternidade e Cirurgia Nossa...,"{'id': 2, 'tipo': 'Monitoria'}",2025-03-21 00:00:00,LIBREXIA-STROKE,Monitoria,Pendente
10326,185,BIOTEST- Tricovid - 1001,1001,998,BIOTEST- Tricovid - 1001 - Maternidade e Cirur...,"{'id': 2, 'tipo': 'Monitoria'}",2025-03-24 00:00:00,BIOTEST- Tricovid - 1001,Monitoria,Pendente
11694,371,EXHALE-2,AR-DEX-22-01,None,EXHALE-2 - Maternidade e Cirurgia Nossa Senhor...,"{'id': 2, 'tipo': 'Monitoria'}",2025-03-25 00:00:00,EXHALE-2,Monitoria,Pendente
10247,187,BIOTEST- Tricovid - 1001,1001,998,BIOTEST- Tricovid - 1001 - Santa Casa de Santos,"{'id': 2, 'tipo': 'Monitoria'}",2025-03-31 00:00:00,BIOTEST- Tricovid - 1001,Monitoria,Pendente
12195,491,GLORIA,OBI-822-011,PT491,GLORIA - Maternidade e Cirurgia Nossa Senhora ...,"{'id': 2, 'tipo': 'Monitoria'}",2025-03-31 00:00:00,GLORIA,Monitoria,Pendente
7173,169,LIBREXIA-STROKE,70033093STR3001,STRUCK,LIBREXIA-STROKE - Maternidade e Cirurgia Nossa...,"{'id': 2, 'tipo': 'Monitoria'}",2025-04-02 00:00:00,LIBREXIA-STROKE,Monitoria,Pendente
15176,1492,EXHALE-4,AR-DEX-22-03,None,EXHALE-4 - Santa Casa de Santos,"{'id': 2, 'tipo': 'Monitoria'}",2025-04-02 00:00:00,EXHALE-4,Monitoria,Pendente
15177,1492,EXHALE-4,AR-DEX-22-03,None,EXHALE-4 - Santa Casa de Santos,"{'id': 2, 'tipo': 'Monitoria'}",2025-04-02 00:00:00,EXHALE-4,Monitoria,Pendente
5364,115,VICTORION 2 - PREVENT,CKJX839B12302,CKJX839B12302,VICTORION 2 - PREVENT - Leforte HMCG,"{'id': 2, 'tipo': 'Monitoria'}",2025-04-04 00:00:00,VICTORION 2 - PREVENT,Monitoria,Pendente
12246,584,FREXALT,EFC17919,None,FREXALT - Leforte HMCG,"{'id': 2, 'tipo': 'Monitoria'}",2025-04-08 00:00:00,FREXALT,Monitoria,Pendente


In [30]:
def enviar_emails(monitorias_pendentes_df):
    global envio_para
    try:
        if monitorias_pendentes_df is None or monitorias_pendentes_df.empty:
            print("Não há visitas nos próximos dias para enviar por e-mail.")
            return

        tabela_html = monitorias_pendentes_df[['Estudo/Centro', 'Monitoria', 'Data Estimada', 'Status']].to_html(
            index=False, escape=False, justify="left", border=0, classes="table"
        )

        msg = MIMEMultipart("alternative")
        msg['From'] = email_usuario
        msg['To'] = email_usuario
        msg['Subject'] = "Visitas de Monitoria Pendentes"

        # Validação da lista de destinatários
        if isinstance(envio_para, list):
            # Expressão regex para validar e-mails
            email_regex = r'^[\w\.-]+@[\w\.-]+\.\w+$'
            envio_para = [email.strip() for email in envio_para if email.strip() and re.match(email_regex, email.strip())]
        else:
            envio_para = []

        # Verifica se há destinatários antes de enviar
        destinatarios = [email_usuario] + envio_para
        if not destinatarios:
            print("Nenhum destinatário válido encontrado. O e-mail não foi enviado.")
            return

        # Somente adiciona Bcc se houver destinatários válidos
        if envio_para:
            msg['Bcc'] = ', '.join(envio_para)

        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <p>Olá,</p>
                <p>Segue abaixo lista com visitas de monitoria programadas para os pr&oacute;ximos dias.</p>
                {tabela_html}
                <p>Este email é gerado automaticamente a partir de informações inseridas na Polo Trial.</p>
                <p>Qualquer dúvida, por favor, contate o <strong><span style="text-decoration: underline;">time BI - SVRI</span></strong>.</p>
            </body>
        </html>
        """
        msg.attach(MIMEText(body, 'html'))

        with smtplib.SMTP(smtp_server, email_port) as server:
            server.starttls()
            server.login(email_usuario, email_senha)
            # Envio corrigido para passar apenas destinatários válidos
            server.sendmail(email_usuario, destinatarios, msg.as_string())

        print("E-mail de visitas pendentes enviado com sucesso!")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

# Chamada da função
enviar_emails(monitorias_pendentes_df)

E-mail de visitas pendentes enviado com sucesso!
